In [2]:
from selenium import webdriver

from selenium.webdriver.edge.service import Service

from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup
import time
import concurrent.futures

In [6]:
def get_html(url):
    options = Options()
    options.add_argument("--headless")
    # WebDriverを起動
    edge_path = "C:/Users/dev/programming/nitac-ai-club-platform/msedgedriver.exe"
    service = Service(edge_path)
    driver = webdriver.Edge(options, service)
    # driver.get("http://google.com")
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    return html

In [3]:
# url = "https://www.kaggle.com/yuuthk/code"
url = "https://www.kaggle.com/rodneybook/code"

In [4]:
html = get_html(url)
soup = BeautifulSoup(html, "html.parser")

In [5]:
html

'<html lang="en"><head>\n  <title>Rodney | Contributor |  Kaggle</title>\n  <meta charset="utf-8">\n    <meta name="robots" content="index, follow">\n  <meta name="description" content="Kaggle Notebooks profile for Rodney">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">\n  <meta name="theme-color" content="#008ABC">\n  <script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.ja.X_IdA0U20Ug.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/am=AACA/rs=AHpOoo-l9iqDK0NyhuLj6M9spzS9xkG6Iw/cb=gapi.loaded_0?le=scs" nonce="" async=""></script><script nonce="" type="text/javascript">\n    window["pageRequestStartTime"] = 1728838400698;\n    window["pageRequestEndTime"] = 1728838400711;\n    window["initialPageLoadStartTime"] = new Date().getTime();\n  </script>\n  <script nonce="" id="gsi-client" src="https://accounts.google.com/gsi/client" async="" defer=""></script>\n  <script nonce="">window.KAGGLE_JUPYTERLAB_PATH = "/static/jl

In [6]:
li_elements = soup.find_all(
    "li",
    # class_="MuiListItem-root MuiListItem-gutters MuiListItem-divider ",
    class_="MuiListItem-root MuiListItem-gutters MuiListItem-divider sc-hDcvty cBatpr css-1nkzj85",
)
# li_elements = soup.find_all(
#     "li",
#     class_="MuiListItem-root MuiListItem-gutters MuiListItem-divider sc-hDcvty cBatpr css-1nkzj85",
# )
prefix = "https://www.kaggle.com"
for li in li_elements:
    links = li.find_all("a")
    # print(li)
    # print([str(link.get('href'))[:] for link in links])
    # print(len(links))
    print(prefix + links[0].get("href"))

https://www.kaggle.com/code/rodneybook/yolo-v5-raw2hns-test
https://www.kaggle.com/code/rodneybook/copy-predict-result
https://www.kaggle.com/code/rodneybook/annotation-visualization
https://www.kaggle.com/code/rodneybook/notebooke
https://www.kaggle.com/code/rodneybook/5th-gld-v7debugdone-all-data-sc
https://www.kaggle.com/code/rodneybook/nirsoil-data-r2py


In [7]:
def get_public_code_url_list(user_name):
    url = f"https://www.kaggle.com/{user_name}/code"
    html = get_html(url)
    soup = BeautifulSoup(html, "html.parser")
    user_name_p = soup.find("p", class_="sc-hfvVTD cZmHWZ")
    if user_name_p is None:
        raise ValueError("user_name is not found")
    li_elements = soup.find_all(
        "li",
        class_="MuiListItem-root MuiListItem-gutters MuiListItem-divider sc-hDcvty cBatpr css-1nkzj85",
    )
    prefix = "https://www.kaggle.com"
    public_code_url_list = []
    for li in li_elements:
        links = li.find_all("a")

        public_code_url_list.append(prefix + links[0].get("href"))

    return public_code_url_list

In [8]:
# url_list = get_public_code_url_list("yuuthk")
public_code_url_list = get_public_code_url_list("rodneybook")

In [9]:
def get_competition_url_and_best_score_from_code_url(url):
    prefix = "https://www.kaggle.com"

    html = get_html(url)
    soup = BeautifulSoup(html, "html.parser")

    # div_if_no_score_available = soup.find("div", class_="sc-dLVkIh hPYerM")
    score_div_previous_sibling = soup.find(
        "div",
        class_="sc-epPVmt",
        #   class_="sc-epPVmt sc-gXwEoq ktnkNI jDdPdJ",
        string="Best Score",
    )
    # print(score_div_previous_sibling)
    # print(div_if_no_score_available, div_if_score_available)
    if score_div_previous_sibling is None:
        raise ValueError("no score available")
    elif score_div_previous_sibling is not None:
        score = float(score_div_previous_sibling.next_sibling.get_text().split(" ")[0])

        competition_relative_url = soup.find("a", class_="sc-pFPEP dCTTgx").get("href")

        return prefix + competition_relative_url, score
    else:
        assert False

In [10]:
def process_url(url):
    try:
        competition_url, score = get_competition_url_and_best_score_from_code_url(url)
        return (url, competition_url, score)
    except ValueError as e:
        return (url, None, None)


# 並列処理を実行
with concurrent.futures.ThreadPoolExecutor() as executor:
    res = executor.map(process_url, public_code_url_list)
print([r for r in res])

[('https://www.kaggle.com/code/rodneybook/yolo-v5-raw2hns-test', None, None), ('https://www.kaggle.com/code/rodneybook/copy-predict-result', None, None), ('https://www.kaggle.com/code/rodneybook/annotation-visualization', None, None), ('https://www.kaggle.com/code/rodneybook/notebooke', None, None), ('https://www.kaggle.com/code/rodneybook/5th-gld-v7debugdone-all-data-sc', None, None), ('https://www.kaggle.com/code/rodneybook/nirsoil-data-r2py', None, None)]


In [26]:
class Score:
    def __init__(self, code_url: str, competition_url: str, score: float):
        self.code_url = code_url
        self.competition_url = competition_url
        self.score = score

    def __repr__(self):
        return f"BestScore(url={self.code_url}, competition_url={self.competition_url}, score={self.score})"


def get_score(url):
    prefix = "https://www.kaggle.com"

    html = get_html(url)
    soup = BeautifulSoup(html, "html.parser")

    # div_if_no_score_available = soup.find("div", class_="sc-dLVkIh hPYerM")
    score_div_previous_sibling = soup.find(
        "div",
        class_="sc-epPVmt",
        #   class_="sc-epPVmt sc-gXwEoq ktnkNI jDdPdJ",
        string="Best Score",
    )
    # print(score_div_previous_sibling)
    # print(div_if_no_score_available, div_if_score_available)
    if score_div_previous_sibling is None:
        raise ValueError("no score available")
    elif score_div_previous_sibling is not None:
        score = float(score_div_previous_sibling.next_sibling.get_text().split(" ")[0])

        competition_relative_url = soup.find("a", class_="sc-pFPEP dCTTgx").get("href")

        return Score(url, prefix + competition_relative_url, score)

    else:
        assert False

In [21]:
def get_best_score_list(public_code_url_list):
    def process_url(url):
        try:
            score = get_score(url)
            return score
        except ValueError as e:
            return None

    # 並列処理を実行
    with concurrent.futures.ThreadPoolExecutor() as executor:
        res = executor.map(process_url, public_code_url_list)

    score_list: list[Score | None] = list(res)

    # score_listの各要素の中でのcompetition_listを調べ、重複したものがあれば、scoreが最大のもののみを一つ選ぶようにする
    competition_url_to_score = {}
    for score in score_list:
        if score is None:
            continue
        if score.competition_url not in competition_url_to_score:
            competition_url_to_score[score.competition_url] = score
        else:
            if score.score > competition_url_to_score[score.competition_url].score:
                competition_url_to_score[score.competition_url] = score

    return list(competition_url_to_score.values())

In [24]:
public_code_url_list = get_public_code_url_list("yuuthk")
# public_code_url_list = get_public_code_url_list("rodneybook")

In [27]:
get_best_score_list(public_code_url_list)

[BestScore(url=https://www.kaggle.com/code/yuuthk/ai-titanic-problem, competition_url=https://www.kaggle.com/competitions/titanic, score=0.75119), BestScore(url=https://www.kaggle.com/code/yuuthk/notebook1cca944ded, competition_url=https://www.kaggle.com/competitions/titanic, score=0.7488)]


[BestScore(url=https://www.kaggle.com/code/yuuthk/ai-titanic-problem, competition_url=https://www.kaggle.com/competitions/titanic, score=0.75119)]

In [7]:
def is_unknown_page(soup):
    unknown_page = soup.find(
        "h6", class_="sc-dkjaqt eCdyAW", string="We can't find that page."
    )
    return unknown_page is not None

In [26]:
search_word = "cat"

search_url = f"https://www.kaggle.com/competitions?searchQuery={search_word}"


html = get_html(search_url)
soup = BeautifulSoup(html, "html.parser")

soup
# print(is_unknown_page(soup))

<html lang="en"><head>
<title>Kaggle Competitions</title>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="Kaggle is the world’s largest data science community with powerful tools and resources to help you achieve your data science goals." name="description"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<meta content="#008ABC" name="theme-color"/>
<script async="" nonce="" src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.ja.X_IdA0U20Ug.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/am=AACA/rs=AHpOoo-l9iqDK0NyhuLj6M9spzS9xkG6Iw/cb=gapi.loaded_0?le=scs"></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1728903627218;
    window["pageRequestEndTime"] = 1728903627222;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
<script async="" defer="" id="gsi-client" nonce="" src="https://accounts.google.com/gsi/client"></script>
<scrip

In [58]:
competition_url = "https://www.kaggle.com/competitions/dogs-vs-cats"

if competition_url[-1] == "/":
    competition_url = competition_url[:-1]

# search_url = f"https://www.kaggle.com/competitions?searchQuery=cat"

search_url = (
    f"https://www.kaggle.com/competitions?searchQuery={competition_url.split("/")[-1]}"
)

html = get_html(search_url)
soup = BeautifulSoup(html, "html.parser")

# print(is_unknown_page(soup))


In [59]:
found_a_list = [
    e.children.__next__().children.__next__()
    for e in soup.find_all(
        "li",
        class_=lambda x: x
        and "MuiListItem-root" in x
        and "MuiListItem-gutters" in x
        and "sc-hDcvty" in x
        and "cBatpr" in x,
    )
]
print(found_a_list)

found_url_list = [
    a
    for a in found_a_list
    if f"https://www.kaggle.com{a.get('href')}" == competition_url
]
found_url_list[0].find("img").get("src")


[<a aria-label="Home" class="sc-llIhno jmYDJg" data-click-log-id="home" href="/" role="link" tabindex="0"><div class="sc-dCKixA dLkNUm"><div class="sc-giOWAb edTbaA"><div class="sc-kuDXwa hUWEl"><span aria-hidden="true" aria-label="Home" class="google-symbols notranslate MuiIcon-root MuiIcon-fontSizeMedium sc-AmOMz faPwmD notranslate sc-cmZgWc dYClmX css-1jgtvd5" title="Home">explore</span></div></div><div class="sc-epPVmt sc-gmSqAz ktnkNI iIMFIV"><p class="sc-fpSrms hezKvo" style="padding-left: 4px;">Home</p></div></div></a>, <a aria-current="page" aria-label="Competitions" class="sc-llIhno jmYDJg" data-click-log-id="competitions" href="/competitions" role="link" tabindex="0"><div class="sc-dCKixA dLkNUm"><div class="sc-giOWAb edTbaA"><div class="sc-kuDXwa hUWEl"><span aria-hidden="true" aria-label="Competitions" class="google-symbols notranslate MuiIcon-root MuiIcon-fontSizeMedium sc-AmOMz imBOkQ notranslate sc-cmZgWc dYClmX css-1jgtvd5" title="Competitions">emoji_events</span></div>

'https://storage.googleapis.com/kaggle-competitions/kaggle/3362/logos/thumb76_76.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1728990782&Signature=j1IVoKc4ZkiLWnOpbWhmoX%2Fgx%2FIVOs6b6ANwjFRa5JTgcu2Xhg4IMdosSDOy0tNEzlXzLjlpS%2FkCXi4F%2B9ec3Dg5eNPg2p8xOYT0Zf9mDU5rVc6xkCwqYgvyEOLWtKugGHK1tSVEoeyTFf1fUIKslpK%2BbPDTSFgg0K%2F4kVF%2B2IlbEEKsNLRpormHlFvZOGepTu4SWSsD8yBGIJRjv%2Fzne1j%2FRCMPst4AF0fBz7YcZr9QhGKNq%2FMI%2BTDznqL%2F4kGW0PIkQZYKJdUL%2FxTIssmUGUWGty1X92oQiDinmXonmQQoCdknYQQcvTNldHDAznBTbuNhA2LEVv0a2b9UmFm5Eg%3D%3D'

In [87]:
def get_competition_picture(competition_url):
    if competition_url[-1] == "/":
        competition_url = competition_url[:-1]

    search_url_base = f"https://www.kaggle.com/competitions?searchQuery={competition_url.split("/")[-1]}"

    html = get_html(search_url_base)
    soup = BeautifulSoup(html, "html.parser")

    found_a_list = [
        e.children.__next__().children.__next__()
        for e in soup.find_all(
            "li",
            class_=lambda x: x
            and "MuiListItem-root" in x
            and "MuiListItem-gutters" in x
            and "sc-hDcvty" in x
            and "cBatpr" in x,
        )
    ]

    found_url_list = [
        a
        for a in found_a_list
        if f"https://www.kaggle.com{a.get('href')}" == competition_url
    ]

    if len(found_url_list) == 0:
        nav_list = soup.find_all(
            "span", class_=lambda x: x and "MuiTouchRipple-root" in x
        )
        for nav_child in nav_list:
            nav = nav_child.parent
            if nav.get_text().isdigit:
                next_page_url = f"{search_url_base}&page={nav.get_text()}"

                html = get_html(next_page_url)
                soup = BeautifulSoup(html, "html.parser")

                found_a_list = [
                    e.children.__next__().children.__next__()
                    for e in soup.find_all(
                        "li",
                        class_=lambda x: x
                        and "MuiListItem-root" in x
                        and "MuiListItem-gutters" in x
                        and "sc-hDcvty" in x
                        and "cBatpr" in x,
                    )
                ]

                found_url_list = [
                    a
                    for a in found_a_list
                    if f"https://www.kaggle.com{a.get('href')}" == competition_url
                ]

                if len(found_url_list) > 0:
                    break

    url = found_url_list[0].find("img").get("src")
    if url[0] == "/":
        url = "https://www.kaggle.com" + url
    return url


In [ ]:
get_competition_picture("https://www.kaggle.com/competitions/flower-classification")


<span class="MuiTouchRipple-root css-w0pj6f"></span>
aa
